In [1]:
COMPANIES_NEWS_PATH =  r'D:\Projects\Quantum Bits\News-Impact-on-Stock-Prices-\Project_data\output\campanies_news.csv'

In [2]:
from urllib.request import urlopen , Request
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
def get_news(tickers: list):
    """
    Fetches recent news headlines for the given list of stock tickers from Finviz.

    Args:
    tickers (list): A list of stock ticker symbols (e.g., ['AAPL', 'GOOGL']).

    Returns:
    pd.DataFrame: A DataFrame containing the ticker symbol, date, time, and headline of the news.
    """
    finviz_url = 'https://finviz.com/quote.ashx?t='  # Base URL for Finviz stock pages
    news_tables = {}  # Dictionary to store news tables for each ticker
    parsed_data = []  # List to store parsed news data

    # Loop through each ticker to fetch news
    for ticker in tickers:
        url = finviz_url + ticker  # Construct URL for the specific stock ticker
        
        # Send a request to the URL with a user-agent header to avoid being blocked
        req = Request(url, headers={'user-agent': 'google'})
        response = urlopen(req)  # Get the response
        
        # Parse the HTML content of the page
        html = BeautifulSoup(response, 'html.parser')
        
        # Find the news table by its ID and store it in the dictionary
        news_table = html.find('table', {'id': 'news-table'})
        news_tables[ticker] = news_table

    # Loop through each ticker's news table to extract headlines and timestamps
    for ticker, news_table in news_tables.items():
        for rows in news_table.findAll('tr'):  # Iterate through each row in the table
            title = rows.a.text.strip()  # Extract the headline text
            date_time_data = rows.td.text.strip().split(' ')  # Extract date and time
            
            # Check if only time is available (no date)
            if len(date_time_data) == 1:
                time = date_time_data[0]
            else:
                date = date_time_data[0]  # Extract the date
                time = date_time_data[1]  # Extract the time
                
            # Append the parsed data to the list
            parsed_data.append([ticker, date, time, title])
    
    # Convert the parsed data into a DataFrame
    df = pd.DataFrame(parsed_data, columns=['Ticker', 'Date', 'Time', 'Headline'])
    
    return df  # Return the DataFrame


In [ ]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans and formats the news DataFrame by removing special characters and converting text to lowercase.

    Args:
    df (pd.DataFrame): The DataFrame containing scraped news headlines.

    Returns:
    pd.DataFrame: The cleaned and formatted DataFrame.
    """
    # Remove special characters from the 'Headline' column and convert text to lowercase
    df['Headline'] = df['Headline'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x.lower()))
    
    # Optional: Convert 'Date' column to a standard date format
    df['Date'] = df['Date'].apply(lambda x: x.replace('Today', pd.to_datetime('today').strftime('%b-%d-%y')))

    return df

In [5]:
tickers = ['DELL' , 'AMZN' , 'META' , 'NVDA' , 'AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NFLX', 'INTC', 'AMD', 'BABA', 'ORCL']

df = get_news(tickers)

In [6]:
df

,Ticker,Date,Time,Headline
0,DELL,Today,10:49PM,Dell Technologies Inc. (DELL): An Undervalued ...
1,DELL,Today,10:30AM,Dell Technologies Inc. (DELL): This AI Stock I...
2,DELL,Aug-30-24,04:40PM,"These Stocks Moved the Most Today: Intel, Supe..."
3,DELL,Aug-30-24,04:36PM,Heard on the Street: Dell Shows AI Isn't a Los...
4,DELL,Aug-30-24,04:17PM,AI Computer Sales Fuel Dell's Better-Than-Expe...
...,...,...,...,...
1295,ORCL,Jul-17-24,09:59AM,Oracle (ORCL) Cloud Enhances Ackermans' Retail...
1296,ORCL,Jul-16-24,11:13AM,3 Machine Learning Stocks That Could Grow Your...
1297,ORCL,Jul-16-24,09:00AM,Oracle Corporation (ORCL) is Attracting Invest...
1298,ORCL,Jul-16-24,05:40AM,Analyst Says These 10 AI Stocks Have More Upsi...


# Data Cleaning

In [7]:
df.isna().sum()

Ticker      0
Date        0
Time        0
Headline    0
dtype: int64

In [9]:
df.head()

,Ticker,Date,Time,Headline
0,DELL,Today,10:49PM,Dell Technologies Inc. (DELL): An Undervalued ...
1,DELL,Today,10:30AM,Dell Technologies Inc. (DELL): This AI Stock I...
2,DELL,Aug-30-24,04:40PM,"These Stocks Moved the Most Today: Intel, Supe..."
3,DELL,Aug-30-24,04:36PM,Heard on the Street: Dell Shows AI Isn't a Los...
4,DELL,Aug-30-24,04:17PM,AI Computer Sales Fuel Dell's Better-Than-Expe...


In [10]:
cleaned_df = clean_data(df)
cleaned_df.head()

,Ticker,Date,Time,Headline
0,DELL,Sep-01-24,10:49PM,dell technologies inc dell an undervalued tech...
1,DELL,Sep-01-24,10:30AM,dell technologies inc dell this ai stock is tr...
2,DELL,Aug-30-24,04:40PM,these stocks moved the most today intel super ...
3,DELL,Aug-30-24,04:36PM,heard on the street dell shows ai isnt a loss ...
4,DELL,Aug-30-24,04:17PM,ai computer sales fuel dells betterthanexpecte...


# save the data

In [11]:
cleaned_df.to_csv(COMPANIES_NEWS_PATH , index=False)